In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn.functional as F # import convolution functions like Relu

## CPU oder GPU?
Torch erlaubt das Ausführen (vor allem der aufwändigen Lernphase) des künstlichen neuronalen Netzes auf CPU oder GPU. Ob die GPU zur Verfügung steht hängt von dem installierten Python-Paket und der Unterstützung der Grafikkarte durch Torch ab.

In [4]:

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


## Knünstliches Neuronale Netzwerke in Torch
In Torch werden neuronale Netzwerke wie durch Python-Klassen repräsentiert, die von der Klasse nn.Module erben.
Diese beinhalten die Netztopologie sowie die Berechnungsart.

In [5]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

## Training des Netzes
Das Training eines Netzes definiert man am besten als eigene Funktion. Das Training folgt hierbei dem bereits aus der Vorlesung bekannten Backpropagation-Verfahren. Die Art der Optimierung, der ```optimizer```, kann durch verschiedene bereits implementierte Optimierer bestimmt werden. Siehe hierzu auch den Aufruf in der ```main()``` Methode 

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

## Testing
Auch das Testing ist am besten als Funktion zu kapseln. Hierbei wird das aktuelle ```model```, also das aktuelle künstliche neuronale Netz mit den Testdaten geprüft. Diese dürfen nicht Teil des Trainings sein. 

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## Die Hauptroutine
Hier passiert alles:
1. Die MNIST-Daten werden geladen und in Trainings- und Testdaten aufgeteilt
2. Die Lernparameter und Optimierungsstrategien werden festgelegt
3. Das Lernen erfolgt, gefolgt von einer Testphase pro Epoche.
4. Das Netz wird abgespeichert (so kann es schnell wieder geladen werden, ohne dass ein erneutes Training erfolgt)

In [8]:
def main():
    # Download training data from open datasets.
    training_data = datasets.MNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor(),
    )

    # Download test data from open datasets.
    test_data = datasets.MNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor(),
    )

    batch_size = 64

    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    for X, y in test_dataloader:
        print(f"Shape of X [N, C, H, W]: {X.shape}")
        print(f"Shape of y: {y.shape} {y.dtype}")
        break

    model = NeuralNetwork().to(device)
    print(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    
    epochs = 10
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test(test_dataloader, model, loss_fn)
    print("Done!")

    torch.save(model.state_dict(), "model.pth")
    print("Saved PyTorch Model State to model.pth")

    model = NeuralNetwork().to(device)
    model.load_state_dict(torch.load("model.pth"))

    classes = test_data.classes

In [9]:
main()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 2.312955  [   64/60000]
loss: 0.745162  [ 6464/60000]
loss: 0.385018  [12864/60000]
loss: 0.355368  [19264/60000]
loss: 0.219450  [25664/60000]
loss: 0.316986  [32064/60000]
loss: 0.227706  [38464/60000]
loss: 0.389943  [44864/60000]
loss: 0.348441  [51264/60000]
loss: 0.270187  [57664/60000]
Test Error: 
 Accuracy: 93.3%, Avg loss: 0.218695 

Epoch 2
-------------------------------
loss: 0.158646  [   64/60000]
loss: 0.195221  [ 6464/60000]
loss: 0.102797  [12864/6